 # Подготовка окружения

In [2]:
import opendatasets as od
import pandas as pd
import time
from pathlib import Path

## Скачиваем датасет

In [5]:
dataset_path = Path('genius-song-lyrics-with-language-information', 'song_lyrics.csv')
if not dataset_path.is_file():
    od.download('https://www.kaggle.com/datasets/carlosgdcj/genius-song-lyrics-with-language-information')

Dataset URL: https://www.kaggle.com/datasets/carlosgdcj/genius-song-lyrics-with-language-information


 64%|██████▍   | 1.96G/3.04G [04:15<01:57, 9.87MB/s] 

In [ ]:
# Количество ядер
!nproc

2


# Задача 4

## Вариант "в лоб"

Здесь показана реализация с помощью пандаса

In [5]:
# Неправильно, упадёт из-за нехватки памяти
# df = pd.read_csv(dataset_path)

## Правильное решение

Обработка файла по частям. Как будет видно мультипроцессорная обработка не всегда самая быстрая, так как у нас появляются дополнительные накладные расходы на мультипоточную реализацию. Но такая реализация позволяет считать, что нельзя посчитать на одном компьютере, хоть и с накладными расходами.

А функции map и reduce позволяют делать код проще и масштабируемей.

###  Считывем с помощью пандас

In [6]:
# Вариант с циклом
chunksize = 100_000
df_chunks = pd.read_csv(
    dataset_path,
    chunksize=chunksize)

start_time = time.time()
sizes = []
for chunk in df_chunks:
    sizes.append(chunk[(chunk.year == 2022) & (chunk.views > 5000)].shape[0])
print(sum(sizes))
print(f'Время выполнения: {round(time.time() - start_time, 2)} секунд.')

2650
Время выполнения: 224.31 секунд.


In [7]:
# Вариант с map и reduce
from functools import reduce


def count_songs(chunk):
    return chunk[(chunk.year == 2022) & (chunk.views > 5000)].shape[0]


chunksize = 100_000
df_chunks = pd.read_csv(
    dataset_path,
    chunksize=chunksize)

start_time = time.time()
print(reduce(lambda x, y: x + y, map(count_songs, df_chunks,)))
print(f'Время выполнения: {round(time.time() - start_time, 2)} секунд.')

2650
Время выполнения: 217.39 секунд.


### С помощью библиотеки csv

In [10]:
import csv
from functools import reduce


year_col = 3
views_col = 4


def count_songs(row):
    year = row[year_col].strip()
    views = row[views_col].strip()
    return 1 if year == '2022' and int(views) > 5_000 else 0


n = 0
start_time = time.time()
with open(dataset_path, "r", encoding='utf8') as file:
    reader = csv.reader(file)
    print(reduce(lambda x, y: x + y, map(count_songs, reader)))
print(f'Время выполнения: {round(time.time() - start_time, 2)} секунд.')

2650
Время выполнения: 227.21 секунд.
